In [25]:
import sys
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path.cwd()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
os.chdir(r"D:\Visual Studio practice\aviation-chatbot")

print(f"✅ Project root: {PROJECT_ROOT}")

✅ Project root: D:\Visual Studio practice\aviation-chatbot


In [26]:
import psycopg2
from src.config import DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD

def test_connection():
    """Test basic PostgreSQL connection"""
    try:
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
        cursor = conn.cursor()
        cursor.execute("SELECT version();")
        version = cursor.fetchone()
        print("✅ PostgreSQL connection successful!")
        print(f"Version: {version[0]}")
        cursor.close()
        conn.close()
        return True
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return False

# Execute the test
test_connection()

✅ PostgreSQL connection successful!
Version: PostgreSQL 16.11 (Debian 16.11-1.pgdg12+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14+deb12u1) 12.2.0, 64-bit


True

In [27]:
def enable_pgvector():
    """Enable pgvector extension for vector operations"""
    try:
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
        
        # Set autocommit to True for CREATE EXTENSION
        conn.autocommit = True
        cursor = conn.cursor()
        
        # Enable pgvector extension
        print("Enabling pgvector extension...")
        cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")
        print("✅ Extension creation command executed")
        
        # Verify it's enabled
        cursor.execute("""
            SELECT extname, extversion 
            FROM pg_extension 
            WHERE extname = 'vector';
        """)
        result = cursor.fetchone()
        
        if result:
            print(f"✅ pgvector extension enabled successfully!")
            print(f"   Extension: {result[0]}")
            print(f"   Version: {result[1]}")
        else:
            print("❌ pgvector extension not found after creation attempt")
            print("   Checking available extensions...")
            cursor.execute("SELECT * FROM pg_available_extensions WHERE name = 'vector';")
            available = cursor.fetchone()
            if available:
                print(f"   Vector extension is available: {available}")
            else:
                print("   ⚠️ Vector extension NOT available in this PostgreSQL image!")
        
        cursor.close()
        conn.close()
        return True
    except Exception as e:
        print(f"❌ Error enabling pgvector: {e}")
        import traceback
        traceback.print_exc()
        return False

# Execute
enable_pgvector()


Enabling pgvector extension...
✅ Extension creation command executed
✅ pgvector extension enabled successfully!
   Extension: vector
   Version: 0.8.1


True

In [28]:
from src.config import KNOWLEDGE_CHUNKS_TABLE

def create_tables():
    """Create the knowledge_chunks table with vector support"""
    try:
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
        cursor = conn.cursor()
        
        # Execute the table creation SQL
        cursor.execute(KNOWLEDGE_CHUNKS_TABLE)
        conn.commit()
        
        print("✅ Table 'knowledge_chunks' created successfully!")
        
        # Verify table structure
        cursor.execute("""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_name = 'knowledge_chunks';
        """)
        columns = cursor.fetchall()
        
        print("\n📊 Table schema:")
        for col in columns:
            print(f"  - {col[0]}: {col[1]}")
        
        cursor.close()
        conn.close()
        return True
    except Exception as e:
        print(f"❌ Error creating table: {e}")
        return False

# Execute
create_tables()


✅ Table 'knowledge_chunks' created successfully!

📊 Table schema:
  - created_at: timestamp without time zone
  - metadata: jsonb
  - embedding: USER-DEFINED
  - id: integer
  - page_number: integer
  - document_name: character varying
  - content: text


True

In [29]:
import numpy as np

def test_vector_insert():
    """Test inserting a sample chunk with vector embedding"""
    try:
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
        cursor = conn.cursor()
        
        # Create a test embedding (384 dimensions for all-MiniLM-L6-v2)
        test_embedding = np.random.rand(384).tolist()
        
        # Insert test data
        cursor.execute("""
            INSERT INTO knowledge_chunks (content, embedding, document_name, page_number)
            VALUES (%s, %s, %s, %s)
            RETURNING id;
        """, (
            "This is a test chunk for database verification.",
            test_embedding,
            "test_document.pdf",
            1
        ))
        
        inserted_id = cursor.fetchone()[0]
        conn.commit()
        
        print(f"✅ Test chunk inserted successfully with ID: {inserted_id}")
        
        # Verify the insert
        cursor.execute("""
            SELECT id, content, document_name, page_number 
            FROM knowledge_chunks 
            WHERE id = %s;
        """, (inserted_id,))
        
        result = cursor.fetchone()
        print(f"\n📄 Retrieved data:")
        print(f"  ID: {result[0]}")
        print(f"  Content: {result[1]}")
        print(f"  Document: {result[2]}")
        print(f"  Page: {result[3]}")
        
        # Clean up test data
        cursor.execute("DELETE FROM knowledge_chunks WHERE id = %s;", (inserted_id,))
        conn.commit()
        print("\n🗑️ Test data cleaned up")
        
        cursor.close()
        conn.close()
        return True
    except Exception as e:
        print(f"❌ Error during test insert: {e}")
        return False

# Execute
test_vector_insert()


✅ Test chunk inserted successfully with ID: 2

📄 Retrieved data:
  ID: 2
  Content: This is a test chunk for database verification.
  Document: test_document.pdf
  Page: 1

🗑️ Test data cleaned up


True

In [30]:
def show_db_stats():
    """Display current database statistics"""
    try:
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
        cursor = conn.cursor()
        
        # Count total chunks
        cursor.execute("SELECT COUNT(*) FROM knowledge_chunks;")
        total_chunks = cursor.fetchone()[0]
        
        # Count unique documents
        cursor.execute("SELECT COUNT(DISTINCT document_name) FROM knowledge_chunks;")
        unique_docs = cursor.fetchone()[0]
        
        # Get table size
        cursor.execute("""
            SELECT pg_size_pretty(pg_total_relation_size('knowledge_chunks'));
        """)
        table_size = cursor.fetchone()[0]
        
        print("📊 Database Statistics:")
        print(f"  Total chunks: {total_chunks}")
        print(f"  Unique documents: {unique_docs}")
        print(f"  Table size: {table_size}")
        
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"❌ Error getting stats: {e}")

# Execute
show_db_stats()


📊 Database Statistics:
  Total chunks: 0
  Unique documents: 0
  Table size: 1032 kB
